In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.metrics import mean_squared_error

**Definicion de variables**

In [17]:
cant_particiones = 5
ventana = "auctions_2019-04-18-2019-04-20"
ventana2 = "auctions_2019-04-21-2019-04-23"
carpeta_data = "data/"
ruta = carpeta_data + ventana
ruta2 = carpeta_data + ventana2
extension = ".csv"
dia_inicio = 18
dia_inicio_v2 = 21
tiempo_max = 60*60*24*3

In [18]:
reg = PassiveAggressiveRegressor()
ult_por_user = pd.DataFrame(columns=["date", "device_id", "ref_type_id", "source_id"])
test_y = pd.DataFrame(columns=["device_id", "date"])

**Se crea el Dataframe de test**

In [20]:
for i in range(cant_particiones):
    test_y_temp = pd.read_csv(f"{ruta2}-users{i}{extension}", low_memory=False, parse_dates = ['date'], index_col=['date'])
    test_y_temp = test_y_temp.reset_index()
    test_y_temp['date'] = test_y_temp['date'] - pd.Timestamp(f'2019-04-{dia_inicio_v2} 00:00:00')
    test_y_temp['date'] = test_y_temp['date'].transform(lambda x: x.total_seconds())
    test_y_temp = test_y_temp.groupby('device_id').agg({'date' : 'min'}).reset_index()
    test_y_temp = test_y_temp[["device_id", "date"]]
    test_y = pd.concat([test_y, test_y_temp])


**Se entrena el algoritmo incrementalmente con cada uno de los 5 archivos**

In [25]:
for i in range(cant_particiones):
    train_x = pd.read_csv(f"{ruta}-users{i}{extension}", low_memory=False, parse_dates = ['date'], index_col=['date'])
    train_x = train_x.sort_index().reset_index()
    train_x['date'] = train_x['date'] - pd.Timestamp(f'2019-04-{dia_inicio} 00:00:00')
    train_x['date'] = train_x['date'].transform(lambda x: x.total_seconds())
    train_x = train_x.fillna(train_x.mode().iloc[0])
    train_x = train_x.sort_values(["device_id", "date"]).reset_index(drop=True)
    train_x["prox_ap"] = train_x.groupby("device_id")["date"].shift(-1)
    train_x["prox_ap"] = train_x["prox_ap"].fillna(tiempo_max)
    train_x["censored"] = (train_x["prox_ap"] == tiempo_max).transform(int)
    
    train_y = pd.Series(name="prox_ap")
    train_y = train_x["prox_ap"]
    train_x.drop(["prox_ap"], axis=1, inplace=True)

    ultimos = train_x.groupby("device_id").last().reset_index()
    ultimos = ultimos.merge(test_y["device_id"],how="inner")
    ult_por_user = pd.concat([ult_por_user, ultimos], ignore_index=True)
        
    reg.partial_fit(train_x, train_y)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/lib/python3.7/site-packages/ipykernel_

In [42]:
ult_por_user = ult_por_user[["date", "device_id", "ref_type_id", "source_id", "censored"]]

In [44]:
ult_por_user = ult_por_user.merge(test_y["device_id"], how="left")

In [95]:
ult_por_user = ult_por_user.sort_values("device_id").reset_index(drop=True)

In [51]:
test_y = test_y.merge(ult_por_user["device_id"], how="inner")

In [56]:
test_y = test_y.sort_values("device_id").reset_index(drop=True)["date"]

In [60]:
preds = reg.predict(ult_por_user)

In [62]:
mean_squared_error(test_y, preds)**0.5

176398.5782744413

**El error es para 230000 usuarios**

**Hay que probar variar los parametros del regressor y ver que onda**